# Modelos: Seq

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../lib')

import math
import random
from bunch import Bunch

import numpy as np
import pandas as pd

import torch
from   torch               import nn, Tensor
import torch.nn.functional as F
from   torch.utils.data    import DataLoader, Subset
from   torch.optim         import SparseAdam, Adam

import pytorch_common.util as pu
from   pytorch_common.modules.fn import Fn
from   pytorch_common.callbacks  import (
    EarlyStop,
    ReduceLROnPlateau,
    Validation,
    SaveBestModel
)
from pytorch_common.callbacks.output import Logger, MetricsPlotter
from pytorch_common.util import (
    set_device_name,
    get_device,
    LoggerBuilder
)

import model as ml

import util as ut

import data.dataset as ds
import data.plot as pl
import data as dt
from data.plot import describe_num_var

import metric as mt
import metric.discretizer as dr

## Setup

In [3]:
pu.LoggerBuilder().on_console().build()

<RootLogger root (INFO)>

In [4]:
pu.set_device_name('gpu')

In [5]:
pu.get_device()

device(type='cuda', index=0)

In [6]:
cpu = torch.device("cpu")

In [7]:
torch.cuda.is_available()

True

In [8]:
torch.__version__

'1.13.1+cu117'

In [9]:
def set_seed(value):
    random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)

In [10]:
set_seed(42)

In [18]:
params = Bunch(
    epochs               = 20,
    feat_seq_len         = 5,
    max_input_seq_len    = 10,
    id_seq_init          = 1,
    batch_size           = 16,
    n_heads              = 2,
    hidden_state_size    = 100,
    n_transformer_layers = 2,
    dropout              = 0.1,
    padding_emb_position = 0,
)

### Examples

In [ ]:
vocab_size = embedding_weights.shape[0]
emb_size   = embedding_weights.shape[1]

emb = ml.EmbeddingLayerFactory.create_from_weights(embedding_weights).to(pu.get_device())
# emb = torch.nn.Embedding(emb.shape[0], params.emb_size)

pos_emb = ml.PositionalEncoding(emb_size, params.dropout).to(pu.get_device())

encoder = ml.TransformerEncoderFactory.create(
    emb_size          = emb_size,
    n_heads           = params.n_heads,
    hidden_state_size = params.hidden_state_size,
    dropout           = params.dropout,
    n_layers          = params.n_transformer_layers
).to(pu.get_device())

mlp_decoder = ml.LinearUtils.init_weights(
    nn.Linear(params.feat_seq_len*emb_size, vocab_size)
).to(pu.get_device())

softmax = nn.Softmax(dim=1)

In [61]:
input_batch = torch.randint(0, 1000, (params.feat_seq_len, params.batch_size)).to(pu.get_device())

# src: (S, N, E)
input_batch_emb = emb(input_batch) * math.sqrt(emb_size)

# src: (S, N, E)
input_batch_emb = pos_emb(input_batch_emb)

# src_mask: (S, S)
src_mask = ml.generate_square_subsequent_mask(params.feat_seq_len).to(pu.get_device())

# src_key_padding_mask: (N, S)
src_key_padding_mask = (input_batch == params.padding_emb_position).to(pu.get_device()).byte()

src_key_padding_mask

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0',
       dtype=torch.uint8)

In [62]:
input_batch.shape, input_batch_emb.shape, src_mask.shape, src_key_padding_mask.shape

(torch.Size([5, 16]), torch.Size([5, 16, 384]), torch.Size([5, 5]), torch.Size([5, 16]))

In [63]:
output =  encoder(
    input_batch_emb, 
    mask                 = src_mask,
    src_key_padding_mask = src_key_padding_mask.t()
)
output.shape

AssertionError: only bool and floating types of key_padding_mask are supported

In [142]:
output2 = output.view(params.batch_size, -1)
output2.shape

torch.Size([16, 1920])

In [143]:
mlp_output = mlp_decoder(output2)

In [144]:
mlp_output

tensor([[-1.9245,  2.0788,  1.1303,  ..., -0.4343,  5.2884, -3.9192],
        [-1.4585,  0.3012, -0.4134,  ...,  4.4634,  0.9562,  0.0663],
        [ 2.5241,  1.4087, -0.4868,  ...,  2.4178, -3.8388, -1.9056],
        ...,
        [ 1.3086, -4.3454,  0.9448,  ...,  4.4063, -0.7090, -1.4716],
        [-0.5636, -3.1097, -4.2092,  ...,  5.0171,  0.6142, -2.1801],
        [ 1.9427,  0.2638, -0.6399,  ...,  5.3650,  2.7164, -4.3463]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [145]:
softmax_output = softmax(mlp_output)

In [146]:
softmax_output

tensor([[3.5882e-07, 1.9656e-05, 7.6136e-06,  ..., 1.5925e-06, 4.8686e-04,
         4.8823e-08],
        [6.0848e-07, 3.5355e-06, 1.7304e-06,  ..., 2.2703e-04, 6.8066e-06,
         2.7953e-06],
        [3.2290e-05, 1.0585e-05, 1.5902e-06,  ..., 2.9035e-05, 5.5681e-08,
         3.8483e-07],
        ...,
        [8.2145e-06, 2.8780e-08, 5.7097e-06,  ..., 1.8193e-04, 1.0924e-06,
         5.0953e-07],
        [1.4285e-06, 1.1197e-07, 3.7290e-08,  ..., 3.7891e-04, 4.6386e-06,
         2.8369e-07],
        [1.8578e-05, 3.4660e-06, 1.4040e-06,  ..., 5.6918e-04, 4.0271e-05,
         3.4490e-08]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [147]:
pos = torch.argmax(softmax_output[0])
softmax_output[0][pos]

tensor(0.0636, device='cuda:0', grad_fn=<SelectBackward0>)